# Combine workgraphs

## Introduction

There are two ways to combine workgraphs:

- Extending a workgraph by the tasks of another
- Adding a workgraph as a task of another

This tutorial will show both, highlighting the differences.


In [1]:
%load_ext aiida
from aiida import load_profile

load_profile()

Profile<uuid='36f11577e5ee4adf9e4a5f5d62d3ae9b' name='presto'>

In [2]:
from aiida_workgraph import WorkGraph, task

## Extending a workgraph by the tasks of another

We first define a workgraph to add two numbers and multiply the sum by a third.

In [5]:
@task
def add(x, y):
    return x + y


@task
def multiply(x, y):
    return x * y


with WorkGraph("AddMultiply") as add_multiply:
    sum_ = add(x=1, y=2)
    prod = multiply(x=sum_, y=3)

add_multiply.to_html()

Next, we define a second workgraph with a task to generate a random number between 1 and 10. We then unpack the tasks of the first workgraph into the second using the `extend` method. Finally, we assign the random number as the factor of the multiplication task.

In [4]:
@task
def generate_random_number(minimum, maximum):
    import random

    return random.randint(minimum, maximum)


with WorkGraph("AddMultiplyExtension") as add_multiply_extended:
    random_number = generate_random_number(minimum=1, maximum=10)
    add_multiply_extended.extend(add_multiply)
    add_multiply_extended.add_link(
        random_number,
        add_multiply_extended.tasks.multiply.inputs.y,
    )

add_multiply_extended.to_html()

Here we've authored a new workgraph as an extension of the first. The resultant workgraph reveals all tasks, with a clear dependency structure.

## Adding a workgraph as a task of another

Next, we will contrast this against adding the first workgraph as a task of the second. There are two ways to do this:

1. Simply adding one workgraph as a task

In [6]:
with WorkGraph("AddMultiplyComposed") as add_multiply_composed:
    random_number = generate_random_number(minimum=1, maximum=10)
    result = add_multiply(
        inputs={
            "multiply": {
                "y": random_number,
            },
        }
    )

add_multiply_composed.to_html()

2. Adding a graph builder function as a task

In [7]:
@task.graph_builder
def add_multiply_builder(x, y, z):
    wg = WorkGraph("AddMultiply")
    add_task = wg.add_task(add, x=x, y=y, name="add")
    wg.add_task(multiply, x=add_task.outputs.result, y=z)
    return wg


with WorkGraph("AddMultiplyGraphBuilder") as add_multiply_extended:
    add_multiply_extended.add_task(generate_random_number)
    add_multiply_task = add_multiply_extended.add_task(
        add_multiply_builder,
        z=add_multiply_extended.tasks.generate_random_number.outputs.result,
    )

add_multiply_extended.to_html()

Here we've directly added the graph builder of the first workgraph as a task of the second. The resultant workgraph shows the random number generation task but does not show the internal dependency structure of the first workgraph. Instead, it is treated as a black box, with the output of the random number generator being passed to the graph builder input. The internal mechanism of the first workgraph is only made visible at runtime, when the first workgraph is executed.

## Summary

In this tutorial, we have seen how to combine workgraphs in two different ways: by extending a workgraph with the tasks of another, and by adding a workgraph as a black-box task. The choice between these methods depends on whether you want to see the internal structure of the combined workgraph or treat it as a black box, effectively trading off transparency and abstraction. Each method has its use cases, and understanding them will help you design more complex workflows in AiiDA.